# Convert jepg images into tf-records
- For given a directory of jpeg images, generate a tf-record file that stores the jpeg images.
- Refer to [this](https://www.kaggle.com/cdeotte/how-to-create-tfrecords)

## Import libraries

In [1]:
import os
import cv2
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

## Parse imagenet labels

In [2]:
# Read imagenet label info
df = pd.read_csv('../data/imagenet-labels.txt', sep='\t')

# Pares imagenet labels. Map each synset to such synset's info 
label_dict = df.set_index('synset').T.to_dict()

## Setting

In [3]:
old_synsets = ['n02106550', 'n02098105', 'n02113624', 'n02091635', 'n02415577', 'n02086079', 'n02493793', 'n02112706', 'n02102177', 'n02088466', 'n02346627', 'n02417914', 'n02102040', 'n02100583', 'n02364673', 'n02091831', 'n02102973', 'n02097047', 'n07747607', 'n02509815', 'n02113978', 'n02494079', 'n02106030', 'n02096051', 'n02088364', 'n02088094', 'n02097658', 'n02113712', 'n02457408', 'n02488702', 'n02493509', 'n02403003', 'n02396427', 'n02397096', 'n02096585', 'n02137549', 'n02130308', 'n02093256', 'n07749582', 'n02108551', 'n02128385', 'n02123159', 'n02117135', 'n02085936', 'n02092002', 'n02480495', 'n02113186', 'n02110063', 'n02486261', 'n02442845', 'n04536866', 'n02114855', 'n02093859', 'n02412080', 'n02129604', 'n02110958', 'n02107683', 'n02389026', 'n02504458', 'n02447366', 'n02086910', 'n02105505', 'n02108089', 'n02120505', 'n02109047', 'n02087046', 'n02110627', 'n02480855', 'n02119789', 'n02104365', 'n02093991', 'n02090622', 'n09246464', 'n02441942', 'n02437312', 'n02106382', 'n02100735', 'n02094433', 'n02096437', 'n02123394', 'n02096294', 'n02123597', 'n02108000', 'n02108915', 'n02085782', 'n02128925', 'n02098413', 'n02484975', 'n02114548', 'n02492035', 'n02097130', 'n02119022', 'n02089078', 'n02445715', 'n02134084', 'n02112018', 'n02422699', 'n02125311', 'n02095889', 'n02090721', 'n02489166', 'n02111129', 'n02487347', 'n02127052', 'n02107574', 'n02108422', 'n02481823', 'n02423022', 'n02106166', 'n02091467', 'n02132136', 'n02096177', 'n02105056', 'n02105162', 'n02114712', 'n02500267', 'n02111277', 'n02100236', 'n02124075', 'n02086240', 'n02099601', 'n02110341', 'n02326432', 'n02486410', 'n02483708', 'n02089867', 'n02097298', 'n02110806', 'n02066245', 'n02104029', 'n02138441', 'n02443484', 'n02356798', 'n02099849', 'n02077923', 'n02115913', 'n02422106', 'n02071294', 'n02098286', 'n02092339', 'n02101556', 'n02093647', 'n09468604', 'n02110185', 'n02107142', 'n02105641', 'n02094114', 'n02391049', 'n02091134', 'n02107908']

imagenet_dir = '/raid/imagenet/train'
synsets = list(label_dict.keys())

## Functions to generate example prototype

In [4]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def str_feature(value):
    """Returns a string from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [5]:
def serialize_example(feature0, feature1, feature2):
    feature = {
        'image/encoded': _bytes_feature(feature0),
        'image/class/label': _int64_feature(feature1),
        'image/class/synset': _bytes_feature(feature2.encode('utf-8'))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

## Create tf-records

In [ ]:
for synset in synsets:
    
    if synset in old_synsets:
        continue
    
    # File and directory path
    tf_record_filepath = '../data/tfrec/train-{}.tfrec'.format(synset)
    jpeg_dirpath = os.listdir('{}/{}'.format(imagenet_dir, synset))
    
    # Write tf-record files for every synset
    with tf.io.TFRecordWriter(tf_record_filepath) as writer:
        
        # For each image
        for img_filename in jpeg_dirpath:
            
            # Parse image
            img_file_path = '{}/{}/{}'.format(imagenet_dir, synset, img_filename)
            img = cv2.imread(img_file_path)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tostring()
            
            # Create example
            example = serialize_example(
                img, 
                label_dict[synset]['tfrecord_label'],
                synset
            )
            
            # Write example
            writer.write(example)

/home/hpark407/anaconda3/envs/neuro-cartography/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


In [4]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:4 -> device: XLA_GPU device



In [4]:
import keras
print(keras.__version__)

2.3.1


Using TensorFlow backend.
